<a href="https://colab.research.google.com/github/QuangTrongLeo/Animal_Identification_DL/blob/main/Animal_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("antoreepjana/animals-detection-images-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/animals-detection-images-dataset


In [ ]:
import os

print(os.listdir(path))

['test', 'train']
